> **DISTILBERT BASE UNCASED**
>
> ---
>
> In this notebook we develop a neural model starting from a pre-trained **DistilBERT** model. We tokenize `paragraphs` and `questions` using the default tokenizer, then use the pretrained model as first layer for the neural network.

# **0. Preliminary Settings**

At first, we need to clone the repository to get access to the code and use utility functions inside the notebook. The `src` folder is then added to the system path so that the modules can be used inside the notebook.

Then, we use the utility functions in the `src` folder to get the ***train*** and ***validation*** splits, while we discard the ***test*** split as it will be used to evaluate the best model only.

In [ ]:
!git clone https://github.com/giuluck/Gangster-SQuAD

import sys
sys.path.append('Gangster-SQuAD/src')

from dataset import get_dataframes
train_df, val_df, _ = get_dataframes('Gangster-SQuAD/data/training_set.json')

In [ ]:
## TODO: remove
train_df = train_df.iloc[:]
val_df = val_df.iloc[:]

# **1. Preprocessing**

We use **HuggingFace** to start from a pretrained **DistilBERT** model with its own vocabulary and tokenizer.

In [ ]:
%%capture
!pip install transformers

### ***1.1. Tokenization***

**HuggingFace** provides a pretrained *DistilBertTokenizer*, which is quite slow, and a faster *BertWordPieceTokenizer*. In order to exploit both of them, we initially load the pretrained tokenizer, store its data, and use that data to build the faster tokenizer.

Once the tokenizer has been built, we use it to process every record in the dataframe in order to build the dataset used for training and testing purposes. This dataset will be composed by:
- a **list of tokens** structured in this way $$[\mbox{CLS}, \mbox{ctx_tok}_0, ..., \mbox{ctx_tok}_i, ..., \mbox{ctx_tok}_n, \mbox{SEP}, \mbox{qst_tok}_0, ..., \mbox{qst_tok}_j, ..., \mbox{qst_tok}_m, \mbox{SEP}]$$ which will be used as input for the *DistilBERT* model
- a **start** and an **end** integer value representing the indices of the boundary tokens that identify the answer in the text, which will be used as outputs for the *DistilBERT* model
- the **original context** and a **list of indices** representing the offsets, expressed in number of *chars* and not in number of *tokens*, which will be used to retrieve the original part of text in the context given the two outputs

In [ ]:
class ModelInfo():
  def __init__(self, pretrained_model, embedding_dim, max_length, cls_token, sep_token):
    self.pretrained_model = pretrained_model
    self.embedding_dim = embedding_dim
    self.max_length = max_length
    self.cls_token = cls_token
    self.sep_token = sep_token

model_info = ModelInfo('distilbert-base-uncased', embedding_dim=768, max_length=512, cls_token=101, sep_token=102)

In [ ]:
from transformers import DistilBertTokenizer
from tokenizers import BertWordPieceTokenizer

DistilBertTokenizer.from_pretrained(model_info.pretrained_model).save_pretrained('slow_tokenizer/')
tokenizer = BertWordPieceTokenizer('slow_tokenizer/vocab.txt', lowercase=True)

In [ ]:
import pandas as pd
from preprocessing import compute_boundaries

def process_dataframe(df):
  def process_record(record):
    # both context and question gets tokenized
    ctx_tokens = tokenizer.encode(record['context'])
    ctx_ids = ctx_tokens.ids[1:-1]                       # [CLS] and [SEP] tokens are discarded
    qst_tokens = tokenizer.encode(record['question'])
    qst_ids = qst_tokens.ids[1:-1]                       # [CLS] and [SEP] tokens are discarded
    # take all the context start chars then add a final index for the last character
    offsets = [s for s, _ in ctx_tokens.offsets[:-1]] + [len(record['context'])]
    # token boundaries to be used during training are computed
    start_token, end_token = compute_boundaries(offsets, record['start'], len(record['answer']))
    # input, output and utility data are returned to form the dataset
    return [ctx_ids, qst_ids, start_token, end_token, offsets]

  processed_df = pd.DataFrame(
    [[id] + process_record(record) for id, record in df.iterrows()],
    columns = ['id', 'ctx_ids', 'qst_ids', 'start token', 'end token', 'offsets']
  ).set_index(['id'])
  return processed_df.join(df)

train_df = process_dataframe(train_df)
val_df = process_dataframe(val_df)

train_df[['ctx_ids', 'qst_ids', 'start token', 'end token', 'offsets']]

> **DistilBERT** can manage sequences of 512 tokens at most, including one *\[CLS\]* token and two *\[SEP\]* tokens in our case. Thus, we exclude from the training dataset those sentences which exceed this limit. Still, validation sentences can be longer, therefore we will crop them later in the dataset creation.

In [ ]:
excluded_train_ids = set([
  id for (id, ctx_ids), qst_ids in zip(train_df['ctx_ids'].items(), train_df['qst_ids'])
     if len(ctx_ids) + len(qst_ids) > model_info.max_length - 3
])

train_df = train_df[~train_df.index.isin(excluded_train_ids)]
train_df

### ***1.2. Correctness Check***

Once the dataframe is ready, we check that whether our tokenization is good enough to let us retrieve the correct answers from the text or not.

Answers are retrived by:
1. getting the two `start` and `end` (token) boundaries that should be computed by the model
2. converting them into a `start_char` and an `end_char` pair of indices, which represent the boundaries in the original context, using the `indices` list
3. selecting the correct portion of the `context` using these two (char) boundaries and strip the obtained substring

Some of the answers are not correct, but this is due to the fact that the answers given in the dataset contain substrings or variations of the words which are present in the text.

In [ ]:
from preprocessing import retrieve_answer, check_correctness

def retrieving_procedure(rec):
  return retrieve_answer(rec['start token'], rec['end token'], rec['offsets'], rec['context'])

check_correctness(pd.concat((train_df, val_df)), retrieving_procedure)

### ***1.3. Dataset Creation***

We can now prepare the dataset using **Torch** utils for data managing.

* The `SquadDataset` class extends *Torch's Dataset* and allows to get input and output data from the dataframe in a lazy way
> Note that we add the *masks* tensor, which is currently a tensor of ones, that is used by *DistilBERT* to identify which token has to be considered and which one has to be discarded. Indeed, when we will pad the sequences, we will concatenate some *zeros* to this *masks* tensor to represent the padding tokens.

* The `DataLoader`, then, is used to create mini-batches from the dataset and, via the custom function, to pad these mini-batches.

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader

class SquadDataset(Dataset):
  def __init__(self, dataframe, model_info):
    self.dataframe = dataframe
    self.max_len = model_info.max_length
    self.cls_tok = model_info.cls_token
    self.sep_tok = model_info.sep_token
        
  def __getitem__(self, index):
    rec = self.dataframe.iloc[index]
    # retrieving paragraph and question tokens, limiting them to the maximal length
    qst_ids = rec['qst_ids'][:self.max_len-3] + [self.sep_tok]
    ctx_ids = [self.cls_tok] + rec['ctx_ids'][:self.max_len-len(qst_ids)-2] + [self.sep_tok]
    len_ids = len(ctx_ids) + len(qst_ids)
    # contexts and questions are used to build the input tensor
    ctx_ids = torch.tensor(ctx_ids)
    qst_ids = torch.tensor(qst_ids)
    input_ids = torch.cat((ctx_ids, qst_ids))
    input_masks = torch.ones_like(input_ids)
    input_tensor = torch.stack((input_ids, input_masks), dim=0)
    # the input tensor is padded to length 512
    pad_tensor = torch.zeros((2, self.max_len - len_ids), dtype=torch.long)
    input_tensor = torch.cat((input_tensor, pad_tensor), dim=1)
    # an output tensor containing the two outputs is created as well
    output_tensor = torch.tensor([rec['start token'], rec['end token']])
    return input_tensor, output_tensor
  
  def __len__(self):
    return len(self.dataframe)

train_data = SquadDataset(train_df, model_info)
val_data = SquadDataset(val_df, model_info)

input, output = train_data[0]
print('TRAIN DATA')
print('Input:', input.shape)
print('  > ids:', input[0].shape)
print('  > types:', input[1].shape)
print('Output:', output.shape)
print('  > start:', output[0].shape)
print('  > end:', output[1].shape)

print()

input, output = val_data[0]
print('VAL DATA')
print('Input:', input.shape)
print('  > ids:', input[0].shape)
print('  > types:', input[1].shape)
print('Output:', output.shape)
print('  > start:', output[0].shape)
print('  > end:', output[1].shape)

In [ ]:
train_loader = DataLoader(train_data, batch_size=16, shuffle=True, num_workers=4, pin_memory=True)
val_loader = DataLoader(val_data, batch_size=16, num_workers=4, pin_memory=True)

for input, output in train_loader:
  print('Input:', input.shape)
  print('  > ids:', input[:, 0].shape)
  print('  > types:', input[:, 1].shape)
  print('Output:', output.shape)
  print('  > start:', output[:, 0].shape)
  print('  > end:', output[:, 1].shape)
  break

# **2. Neural Model**

**DistilBERT** is a language model and we will use it as an encoder to produce contextual embeddings for our tokens.

In [ ]:
%%capture
!pip install wandb pytorch-lightning

### ***2.1. Baseline***

> In the Baseline the model returns a dictionary with *one output*, the `last_hidden_state`, which has shape $[\mbox{batch_size}, \mbox{sequence_length}, \mbox{embedding_dimension}]$.

In [ ]:
from torch import nn
import torch.nn.functional as F
from transformers import DistilBertModel
import pytorch_lightning as pl

class DistilBertCategorical(pl.LightningModule):
    def __init__(self, model_info=model_info, alpha=0.5):
        super().__init__()
        self.alpha = alpha
        self.encoder = DistilBertModel.from_pretrained(model_info.pretrained_model)
        self.start_fc = nn.Linear(model_info.embedding_dim, 1)
        self.end_fc = nn.Linear(model_info.embedding_dim, 1)
        self.softmax = nn.Softmax(dim=1)
        self.criterion = nn.CrossEntropyLoss()

    def _logits(self, x):
        x = self.encoder(input_ids=x[:, 0], attention_mask=x[:, 1])
        x = x["last_hidden_state"]
        start = self.start_fc(x).squeeze(dim=2)
        end = self.end_fc(x).squeeze(dim=2)
        return start, end

    def forward(self, x):
        start, end = self._logits(x)
        prob_start = self.softmax(start)
        prob_end = self.softmax(end)
        _, start_indices = prob_start.max(dim=1)
        _, end_indices = prob_end.max(dim=1)

        return start_indices, end_indices

    def training_step(self, batch, batch_idx):
        x, y = batch
        pred_start, pred_end = self._logits(x)
        loss_start = self.criterion(pred_start, y[:, 0])
        loss_end = self.criterion(pred_end, y[:, 1])
        self.log('loss_start', loss_start, prog_bar=True)
        self.log('loss_end', loss_end, prog_bar=True)
        loss = self.alpha * loss_start + (1 - self.alpha) * loss_end
        return loss

    def validation_step(self, batch, batch_idx):
        return self.training_step(batch, batch_idx)

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-5)
        return optimizer

### ***2.2 DistilBERT with dot product attention over Hidden Layers and Highway***

> The base model is focused on the `last_hidden_state` which is passed through two linear layers and a softmax function to obtain `prob_start` and `prob_end`. The reason behind this choice lies in the most informative content of the last representation. However this approach is an approximation because we would be excluding a priori the additional information contained in the previous `hidden_states`. With this extended model "we aim to combine
hidden states from previous time steps by weighting them to generate an overall representation that may be able to capture additional semantic and positional patterns" according to Takeuchi et al. In essence the last `k` `hidden_states` are concatenated, hence the output of the model become $[\mbox{batch_size}, \mbox{sequence_length}, \mbox{embedding_dimension}, \mbox{k}]$ and it is passed through a linear layer which weight the `hidden_states` and output a tensor of size $[\mbox{batch_size}, \mbox{sequence_length}, \mbox{embedding_dimension}]$


> The **Highway** network can be used before computing logits and it must improve the performances because it is able to filter the irrelevant information. The highway model computes:

*   $x\_proj = ReLU(W\_proj_x + b\_proj)$
*   $x\_gate = σ(W\_gate_x + b\_gate)$

> It returns: $x\_highway = x\_gate ⊙ x\_proj + (1 − x\_gate) ⊙ x$

In [ ]:
class Highway(nn.Module):
    def __init__(self, in_size, n_layers=1, act=F.relu):
        super(Highway, self).__init__()
        self.n_layers = n_layers
        self.act = act

        self.normal_layer = nn.ModuleList([nn.Linear(in_size, in_size) for _ in range(n_layers)])
        self.gate_layer = nn.ModuleList([nn.Linear(in_size, in_size) for _ in range(n_layers)])

    def forward(self, x):
        for i in range(self.n_layers):
            normal_layer_ret = self.act(self.normal_layer[i](x))
            gate = torch.sigmoid(self.gate_layer[i](x))

            x = torch.add(torch.mul(normal_layer_ret, gate),torch.mul((1.0 - gate), x))
        return x

In [ ]:
# https://web.stanford.edu/class/archive/cs/cs224n/cs224n.1194/reports/default/15737384.pdf

class DistilBertCategoricalWHL(DistilBertCategorical):
    def __init__(self, model_info=model_info, k=1, alpha=0.5, highway=False):
        super(DistilBertCategoricalWHL, self).__init__(model_info)
        self.k = k
        self.highway = highway
        if highway:
            self.highway = Highway(in_size=model_info.embedding_dim * k)
        self.dot_prod_attention = nn.Linear(k * model_info.embedding_dim, model_info.embedding_dim)


    def _logits(self, x):
        x = self.encoder(input_ids=x[:, 0], attention_mask=x[:, 1], output_hidden_states=True)
        x_last_hidden_state = torch.unsqueeze(x["last_hidden_state"], dim=3)
        x_hidden_states = x["hidden_states"]
        if self.k > 1:
            x = torch.cat([x_last_hidden_state, torch.stack([x_hidden_state for x_hidden_state in x_hidden_states[-(self.k - 1):]], dim=3)], dim=3)
        else:
            x = x_last_hidden_state
        x = x.flatten(start_dim = 2, end_dim = 3)
        if self.highway:
            x = self.highway(x)
        x = self.dot_prod_attention(x)
        start = self.start_fc(x).squeeze(dim=2)
        end = self.end_fc(x).squeeze(dim=2)
        return start, end

### ***2.3 DistilBERT with CNN***
> With this extension we would like to achieve a similar goal of the **DistilBERT with dot product attention over Hidden Layers**: aggregate multiple informations of different hidden layers to obtain a more complete representation layer. For this purpose we use a CNN which takes an input with dimension $[\mbox{batch_size}, \mbox{embedding_dimension}, \mbox{sequence_length}, \mbox{4}]$ and outputs a tensor with dimension $[\mbox{batch_size}, \mbox{embedding_dimension}, \mbox{sequence_length}]$. The output is reshaped  so that it can be correctly taken as input from the two linear layers.

In [ ]:
class DistilBertCategoricalCNN(DistilBertCategorical):
    def __init__(self, model_info=model_info, alpha=0.5):
        super(DistilBertCategoricalCNN, self).__init__(model_info)
        self.conv = nn.Conv2d(in_channels=model_info.embedding_dim, out_channels=model_info.embedding_dim, kernel_size=(1, 2), stride=(1, 4))
    
    def _logits(self, x):
        x = self.encoder(input_ids=x[:, 0], attention_mask=x[:, 1], output_hidden_states=True)
        x_last_hidden_state = torch.unsqueeze(x["last_hidden_state"], dim=3)
        x_hidden_states = x["hidden_states"]
        x = torch.cat([x_last_hidden_state, torch.stack([x_hidden_state for x_hidden_state in x_hidden_states[-3:]], dim=3)], dim=3)
        x = x.transpose(1, 2)
        x = self.conv(x).squeeze(dim=3)
        x = x.transpose(1, 2)
        start = self.start_fc(x).squeeze(dim=2)
        end = self.end_fc(x).squeeze(dim=2)
        return start, end

# **3. Training**

In [ ]:
import wandb
from pytorch_lightning.loggers import WandbLogger

# Access to W&B
wandb.login()

wandb_project_name = "SQuAD"
# Automatically assigned if None, duplicates are automatically handled
wandb_run_name = None
# Entity
wandb_entity = 'lomb'

# Create the logger
wandb_logger = WandbLogger(project="SQuAD", entity=wandb_entity, name=wandb_run_name)

In [ ]:
model = DistilBertCategorical()

trainer = pl.Trainer(
    logger=wandb_logger,
    max_epochs=4, precision=16,
    gpus=-1, distributed_backend='ddp',
    checkpoint_callback=False
)

trainer.fit(model, train_loader)

In [ ]:
# Run this to stop the synchronization with the previous run
wandb.finish()

# **4. Results**

In [ ]:
starts, ends = [], []
num_batches = len(val_loader)

# pytorch lightning does not move the model to GPU during evaluation
# so we need to do that by hand for both the model and the inputs
model = model.cuda()
model.eval()

print('Starting Evaluation...')
for idx, (input, _) in enumerate(val_loader):
  if (idx + 1) % 100 == 0:
    print(f'Batch {idx+1:{len(str(num_batches))}}/{num_batches}')
  with torch.no_grad(): 
    s, e = model(input.cuda())
  starts.append(s)
  ends.append(e)

val_df['pred_start'] = [s.item() for ss in starts for s in ss]
val_df['pred_end'] = [e.item() for ee in ends for e in ee]

In [ ]:
def retrieving_procedure(rec):
  return retrieve_answer(rec['pred_start'], rec['pred_end'], rec['offsets'], rec['context'])

# When the prediction (for both 'start' and 'end') points to a padding character 
# then it is placed at the last offset
val_df['length_offsets'] = val_df['offsets'].apply(lambda x: len(x))
val_df['pred_start'] = (val_df['pred_start'] < val_df['length_offsets']) * val_df['pred_start'] + (val_df['pred_start'] >= val_df['length_offsets']) * (val_df['length_offsets'] - 1)
val_df['pred_end'] = (val_df['pred_end'] < val_df['length_offsets']) * val_df['pred_end'] + (val_df['pred_end'] >= val_df['length_offsets']) * (val_df['length_offsets'] - 1)
val_wrong = check_correctness(val_df, retrieving_procedure)
val_wrong

In [ ]:
print('Validation Score:', 1 - len(val_wrong) / len(val_df))